In [ ]:
gcloud auth list

In [ ]:
gcloud config list project

In [ ]:
PROJECT_ID=$(gcloud config get-value project)
export BUCKET_NAME="${PROJECT_ID}-bucket"

In [ ]:
export REGION=us-central1

##Task 1. Create a Cloud Storage bucket

In [ ]:
gcloud services disable dataflow.googleapis.com
gcloud services enable dataflow.googleapis.com

In [ ]:
gsutil mb gs://$PROJECT_ID

##Task 2. Create a BigQuery dataset and table

In [ ]:
export DATASET=sensors_287
export TABLE=temperature_256

In [ ]:
bq mk $DATASET

In [ ]:
bq mk --table \
$PROJECT_ID:$DATASET.$TABLE \
data:string

##Task 3. Set up a Pub/Sub topic

In [ ]:
export TOPIC=sensors-temp-51576

In [ ]:
gcloud pubsub topics create $TOPIC

In [ ]:
gcloud pubsub subscriptions create $TOPIC-sub --topic=$TOPIC

##Task 4. Run a Dataflow pipeline to stream data from Pub/Sub to BigQuery

In [ ]:
# Dataflow -> + Create job from template ->

In [ ]:
# OR
export JOB=dfjob-80190

In [ ]:
gcloud dataflow flex-template run $JOB --region $REGION \
--template-file-gcs-location gs://dataflow-templates-$REGION/latest/flex/PubSub_to_BigQuery_Flex \
--temp-location gs://$PROJECT_ID/temp/ \
--parameters outputTableSpec=$PROJECT_ID:$DATASET.$TABLE,\
inputTopic=projects/$PROJECT_ID/topics/$TOPIC,\
outputDeadletterTable=$PROJECT_ID:$DATASET.$TABLE,\
javascriptTextTransformReloadIntervalMinutes=0,\
useStorageWriteApi=false,\
useStorageWriteApiAtLeastOnce=false,\
numStorageWriteApiStreams=0

In [ ]:
gcloud dataflow jobs run $JOB --gcs-location gs://dataflow-templates-$REGION/latest/PubSub_to_BigQuery --region=$REGION --project=$PROJECT_ID --staging-location gs://$PROJECT_ID/temp --parameters inputTopic=projects/$PROJECT_ID/topics/$TOPIC,outputTableSpec=$PROJECT_ID:$DATASET.$TABLE


##Task 5. Publish a test message to the topic and validate data in BigQuery

In [ ]:
gcloud pubsub topics publish $TOPIC --message '{"data": "73.4 F"}'

In [ ]:
# BigQuery -> + SQL
SELECT * FROM qwiklabs-gcp-01-b48d842fe3af.sensors_287.temperature_256